In [1]:
!pip install resampy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.7 MB/s eta 0:00:00


In [2]:
import sys
print(sys.executable)

/usr/bin/python3


In [3]:
!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio

Dataset URL: https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio
License(s): CC-BY-NC-SA-4.0
 96% 413M/429M [00:03<00:00, 82.5MB/s]
100% 429M/429M [00:04<00:00, 111MB/s] 


In [4]:
!unzip /content/ravdess-emotional-speech-audio.zip

Archive:  /content/ravdess-emotional-speech-audio.zip
  inflating: Actor_01/03-01-01-01-01-01-01.wav  
  inflating: Actor_01/03-01-01-01-01-02-01.wav  
  inflating: Actor_01/03-01-01-01-02-01-01.wav  
  inflating: Actor_01/03-01-01-01-02-02-01.wav  
  inflating: Actor_01/03-01-02-01-01-01-01.wav  
  inflating: Actor_01/03-01-02-01-01-02-01.wav  
  inflating: Actor_01/03-01-02-01-02-01-01.wav  
  inflating: Actor_01/03-01-02-01-02-02-01.wav  
  inflating: Actor_01/03-01-02-02-01-01-01.wav  
  inflating: Actor_01/03-01-02-02-01-02-01.wav  
  inflating: Actor_01/03-01-02-02-02-01-01.wav  
  inflating: Actor_01/03-01-02-02-02-02-01.wav  
  inflating: Actor_01/03-01-03-01-01-01-01.wav  
  inflating: Actor_01/03-01-03-01-01-02-01.wav  
  inflating: Actor_01/03-01-03-01-02-01-01.wav  
  inflating: Actor_01/03-01-03-01-02-02-01.wav  
  inflating: Actor_01/03-01-03-02-01-01-01.wav  
  inflating: Actor_01/03-01-03-02-01-02-01.wav  
  inflating: Actor_01/03-01-03-02-02-01-01.wav  
  inflating: Ac

In [1]:
import numpy as np
import librosa
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D


In [2]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
# Emotions to observe
observed_emotions=['neutral','calm','happy','sad','angry','fearful', 'disgust','surprised']

In [3]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mel_spectrogram = librosa.feature.melspectrogram(y=X, sr=sample_rate).T
    return mel_spectrogram

In [4]:
def load_data(test_size=0.2):
    x, y = [], []
    for file in glob.glob('/content/audio_speech_actors_01-24/Actor_*/*.wav'):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file)
        x.append(feature)
        y.append(emotion)

    x_padded = tf.keras.preprocessing.sequence.pad_sequences(x, dtype='float32')

    return train_test_split(x_padded, y, test_size=test_size, random_state=5)

In [5]:
x_train, x_test, y_train, y_test = load_data()


In [6]:
x_train = np.array([x.reshape((x.shape[0], x.shape[1], 1)) for x in x_train])
x_test = np.array([x.reshape((x.shape[0], x.shape[1], 1)) for x in x_test])

In [7]:
encoder = LabelEncoder()
y_train = tf.keras.utils.to_categorical(encoder.fit_transform(y_train))
y_test = tf.keras.utils.to_categorical(encoder.transform(y_test))

In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(x_train.shape[1], x_train.shape[2], 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(observed_emotions), activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [10]:
model.fit(x_train, y_train, batch_size=32, epochs=50, validation_data=(x_test, y_test), verbose=1)


Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.1992 - loss: 4.4353 - val_accuracy: 0.2118 - val_loss: 1.9684
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 61s 2s/step - accuracy: 0.2881 - loss: 1.8730 - val_accuracy: 0.2500 - val_loss: 1.8964
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.3621 - loss: 1.7594 - val_accuracy: 0.3542 - val_loss: 1.8637
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.4038 - loss: 1.6279 - val_accuracy: 0.3611 - val_loss: 1.9173
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.4558 - loss: 1.4519 - val_accuracy: 0.3681 - val_loss: 2.0055
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.5143 - loss: 1.3565 - val_accuracy: 0.3819 - val_loss: 2.0925
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.5398 - loss: 1.4071 - val_accuracy: 0.4028 - val_loss: 2.0966
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 0.5744 - loss: 1.2319 - val_accuracy: 0.4028 - val_loss:

In [11]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print("CNN Model Accuracy: {:.2f}%".format(accuracy * 100))

CNN Model Accuracy: 52.08%
